In [ ]:
import cv2
import numpy as np
import glob
import yaml
# from IPython.display import display, Markdown

# Configuration
CHECKERBOARD = (7, 9)  # Inner corners for 7x9 checkerboard (rows-1, cols-1)
SQUARE_SIZE = 0.015  # 15mm in meters
CAL_LEFT_PATH = r"left/*.png"  # Use raw string
CAL_RIGHT_PATH = r"right/*.png"  # Use raw string
IMAGE_SIZE = (1280, 720)  # (width, height) - ADDED LINE

# Prepare object points
objp = np.zeros((CHECKERBOARD[0]*CHECKERBOARD[1], 3), np.float32)
objp[:,:2] = np.mgrid[0:CHECKERBOARD[0], 0:CHECKERBOARD[1]].T.reshape(-1,2) * SQUARE_SIZE

def calibrate_camera(image_paths, cam_name):
    objpoints = []
    imgpoints = []
    valid_images = []
    failed_images = []
    
    image_files = sorted(glob.glob(image_paths))
    if not image_files:
        raise ValueError(f"No images found at {image_paths}!")

    print(f"\n=== Processing {cam_name} ===")
    print(f"Total images found: {len(image_files)}")
    
    for idx, fname in enumerate(image_files, 1):
        img = cv2.imread(fname)
        if img is None:
            failed_images.append(fname)
            continue
            
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        ret, corners = cv2.findChessboardCorners(
            gray, CHECKERBOARD,
            cv2.CALIB_CB_ADAPTIVE_THRESH + 
            cv2.CALIB_CB_FAST_CHECK +
            cv2.CALIB_CB_NORMALIZE_IMAGE
        )
        
        if ret:
            corners_subpix = cv2.cornerSubPix(gray, corners, (11,11), (-1,-1),
                (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001))
            
            objpoints.append(objp)
            imgpoints.append(corners_subpix)
            valid_images.append(fname)
        else:
            failed_images.append(fname)

    if not objpoints:
        raise ValueError(f"No valid checkerboard images found for {cam_name}!")

    # Camera calibration
    ret, K, D, rvecs, tvecs = cv2.calibrateCamera(
        objpoints, imgpoints, gray.shape[::-1], None, None,
        flags=cv2.CALIB_ZERO_TANGENT_DIST + cv2.CALIB_FIX_K3 + cv2.CALIB_FIX_K4 + cv2.CALIB_FIX_K5
    )
    
    # Calculate reprojection error
    mean_error = 0
    for i in range(len(objpoints)):
        imgpoints2, _ = cv2.projectPoints(objpoints[i], rvecs[i], tvecs[i], K, D)
        error = cv2.norm(imgpoints[i], imgpoints2, cv2.NORM_L2)/len(imgpoints2)
        mean_error += error
    mean_error /= len(objpoints)
    
    return K, D, objpoints, imgpoints, mean_error, image_files

# Main calibration routine
try:
    print("Starting stereo calibration...")
    K_left, D_left, objpoints_left, imgpoints_left, error_left, image_files_left = calibrate_camera(CAL_LEFT_PATH, "Left Camera")
    K_right, D_right, objpoints_right, imgpoints_right, error_right, image_files_right = calibrate_camera(CAL_RIGHT_PATH, "Right Camera")

    # Stereo calibration
    flags = cv2.CALIB_FIX_INTRINSIC + cv2.CALIB_USE_INTRINSIC_GUESS
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 100, 1e-5)

    ret, K1, D1, K2, D2, R, T, E, F = cv2.stereoCalibrate(
        objpoints_left, imgpoints_left, imgpoints_right,
        K_left, D_left, K_right, D_right,
        IMAGE_SIZE,
        criteria=criteria, flags=flags)

    # Stereo rectification
    R1, R2, P1, P2, Q, _, _ = cv2.stereoRectify(
        K1, D1, K2, D2, 
        IMAGE_SIZE,  # Use fixed image size
        R, T, 
        flags=cv2.CALIB_ZERO_DISPARITY, 
        alpha=0.9
    )

    # Save calibration results
    calibration_data = {
        'image_size': IMAGE_SIZE,
        'K_left': K_left, 'D_left': D_left,
        'K_right': K_right, 'D_right': D_right,
        'R': R, 'T': T,
        'E': E, 'F': F,
        'R1': R1, 'R2': R2,
        'P1': P1, 'P2': P2,
        'Q': Q,
        'error_left': error_left,
        'error_right': error_right,
        'stereo_error': ret
    }

    with open(r'stereo_calibration.yaml', 'w') as f:
        yaml.dump({k: v.tolist() if isinstance(v, np.ndarray) else v 
                  for k,v in calibration_data.items()}, f)

    print("Stereo calibration and rectification successful.")
    print("Results saved to stereo_calibration.yaml")

    # Load the calibration results for printing
    with open(r'stereo_calibration.yaml', 'r') as f:
        data = yaml.safe_load(f)

    def print_matrix(name, matrix):
        print(f"\n{name}:")
        for row in matrix:
            print("  [", end="")
            print(", ".join(f"{x:.8f}" for x in row), end="]\n")

    # Print the Left Camera Calibration Results
    print("=== Left Camera Calibration Results ===")
    print(f"Mean Reprojection Error: {data['error_left']:.5f} px")
    print_matrix("Camera Matrix", data['K_left'])
    print("Distortion Coefficients:")
    print(np.array(data['D_left']).ravel())

    # Print the Right Camera Calibration Results
    print("\n=== Right Camera Calibration Results ===")
    print(f"Mean Reprojection Error: {data['error_right']:.5f} px")
    print_matrix("Camera Matrix", data['K_right'])
    print("Distortion Coefficients:")
    print(np.array(data['D_right']).ravel())

    # Print the Stereo Calibration Results
    print("\n=== Stereo Calibration Results ===")
    print(f"Stereo Reprojection Error: {data['stereo_error']:.5f}")
    print_matrix("Rotation Matrix (R)", data['R'])
    print_matrix("Translation Vector (T)", data['T'])
    print_matrix("Rectification Rotation R1", data['R1'])
    print_matrix("Rectification Rotation R2", data['R2'])
    print_matrix("Projection Matrix P1", data['P1'])
    print_matrix("Projection Matrix P2", data['P2'])
    print_matrix("Disparity-to-Depth Matrix Q", data['Q'])
    print("\nImage Size:")
    print(tuple(data['image_size']))

except Exception as e:
    print(f"Calibration failed: {str(e)}")

=== Left Camera Calibration Results ===
Mean Reprojection Error: 0.02441 px

Camera Matrix:
  [ 1190.26044525, 0.00000000, 640.00000000 ]
  [ 0.00000000, 1184.18390037, 360.00000000 ]
  [ 0.00000000, 0.00000000, 1.00000000 ]
Distortion Coefficients: [ 0.0554087 -0.28       0.         0.         0.       ]

=== Right Camera Calibration Results ===
Mean Reprojection Error: 0.02111 px

Camera Matrix:
  [ 1225.32278763, 0.00000000, 640.00000000 ]
  [ 0.00000000, 1218.45868923, 360.00000000 ]
  [ 0.00000000, 0.00000000, 1.00000000 ]
Distortion Coefficients: [ 0.09020865 -0.42        0.          0.          0.        ]

=== Stereo Calibration Results ===
Stereo Reprojection Error: 40.39357

Rotation Matrix (R):
  [ 0.99990000, -0.00500000, 0.01000000 ]
  [ 0.00480000, 0.99995000, 0.00700000 ]
  [ -0.01010000, -0.00690000, 0.99992000 ]

Translation Vector (T):
  [ -0.17000000 ]
  [ 0.00141493 ]
  [ 0.01626577 ]
Image Size: (1280, 720)
